In [1]:
import requests
import xml.etree.ElementTree as ET
import json
import pandas as pd

In [2]:
def parse_polygons(region_osm_relation_id: int, main_region_osm_relation_id:int, area_level: int):
    
    openstreetmap_api_link = 'https://www.openstreetmap.org/api/'
    openstreetmap_api_version = '0.6/'
    
    # Запрос XML с openstreetmap
    response = requests.get(openstreetmap_api_link + openstreetmap_api_version + '/relation/{0}'.format(str(region_osm_relation_id)))
    if response.status_code == 200:
        # Обработка XML
        root = ET.fromstring(response.text)
    elif response.status_code == 404:
        print('BAD: relation not found')
        return False
    elif response.status_code == 410:
        print('BAD: relation has been deleted')
        return False
    else:
        print('BAD: X3')
        return False

    # Получим наименование области и статус из XML
    if root.find("./relation/tag[@k='name:ru'][@v]") is not None:
        area_ru_name = root.find("./relation/tag[@k='name:ru']").attrib['v']
    elif root.find("./relation/tag[@k='name'][@v]") is not None:
        area_ru_name = root.find("./relation/tag[@k='name']").attrib['v']
    else:
        area_ru_name = ''
        
    if root.find("./relation/tag[@k='name:en'][@v]") is not None:
        area_en_name = root.find("./relation/tag[@k='name:en']").attrib['v']
    else:
        area_en_name = ''

    if root.find("./relation/tag[@k='official_status'][@v]") is not None:
        area_status = root.find("./relation/tag[@k='official_status']").attrib['v'].split(':',1)[1]
    else:
        area_status = ''
        
    if 'городской округ' in area_ru_name and area_status.strip() == '':
        area_status = 'городской округ'
        
    if 'административный округ' in area_ru_name and area_status.strip() == '':
        area_status = 'административный округ'
    
    # получим полигон области
    get_polygon_link = 'http://polygons.openstreetmap.fr/get_geojson.py?id=' + str(region_osm_relation_id) + '&params=0'
    polygon_response = requests.get(get_polygon_link)
    if polygon_response.status_code == 200:
        geo_json = json.loads(polygon_response.text)
    else:
        print('BAD: X3')
        return False

    # MultiPolygon list:
    multi_polygon_list = geo_json['geometries'][0]['coordinates']

    if area_level == 1:
        file_write_param = 'w'
    else:
        file_write_param = 'a'
    
    geojson_list = geo_json['geometries']
    with open('data/{0}_GeoJsonsMultiPolygon.txt'.format(str(main_region_osm_relation_id)), file_write_param) as polygons_fp:
        for geojson in geojson_list:
            polygons_fp.write(  str(area_level) + '\t' + \
                                str(region_osm_relation_id) + '\t' + \
                                area_ru_name + '\t' + \
                                area_en_name + '\t' + \
                                area_status + '\t' + \
                                str(geojson).replace("'",'"') + \
                                '\n'
                             )
            
    # для МО есть актуальная иерархия административного деления
    if area_level == 1 and main_region_osm_relation_id == 51490:

        # Запрос XML relationS с openstreetmap
        response = requests.get(openstreetmap_api_link + openstreetmap_api_version + '/relation/181099'.format(str(region_osm_relation_id)))
        if response.status_code == 200:
            # Обработка XML
            root = ET.fromstring(response.text)
        else:
            return False

        sub_areas_list = []
        for subgroup in root.findall("./relation/member[@role='child']"):
            sub_areas_list.append(int(subgroup.attrib['ref']))

        if len(sub_areas_list) == 0:
            return True
        else:
            area_level += 1
            for sub_area in sub_areas_list:
                parse_polygons(sub_area, main_region_osm_relation_id, area_level)
                
    else:
        sub_areas_list = []
        for subgroup in root.findall("./relation/member[@role='subarea']"):
            sub_areas_list.append(int(subgroup.attrib['ref']))

        if len(sub_areas_list) == 0:
            return True
        else:
            area_level += 1
            for sub_area in sub_areas_list:
                parse_polygons(sub_area, main_region_osm_relation_id, area_level)            

In [3]:
# Идентификатор области, для которой надо получить все полигоны:
# Нужный идентификатор можно найти на сайте http://data.esosedi.org/
main_region_osm_relation_id = 2555133 # Идентификатор МО
area_level = 1

# Запуск процедуры
res = parse_polygons(main_region_osm_relation_id,main_region_osm_relation_id,area_level)